## Discussion: Python defs and the execution model

<br>
university of miami
<br>
REU summer 2022
<br>
Burton Rosenberg.
<br>
<br>last update: May 23, 2022

The Python programming language began around 1990, designed by Guido van Rossum. It is a modern interpreted language supporting a high level of convenience and data abstraction. It competes with Ruby and somewhat with PERL for a universal next-generation language. In the desire to apply Python to scientific problems, the SciPy project provides libraries for highly efficient scientific computation. SciPy is packaged so that it can be installed easily. As a final layer, Jupyter provides a sort of integrated environment of documentation, editing and running Python programs in frameworks called Notebooks.


Documentation on python is available at https://docs.python.org/3/. There are two versions of Python and they are incompatible. The 2.7 is the final version 2. We will use version 3.

### Defines, if-else, scopes and namespaces

There are about four different types of code flow in a program: 
* the normal top to bottom statement by statement; 
* the function call and return; 
* the if-else; 
* the loop; 
* and the exception or catch-throw.


#### Defines

Define functions with the keyword "def" followed by the function name and parameters. 

If the function is one line, follow this by the : and that one line. Else follow this by the : 
and a sequence of lines, all exactly indented.

The Python language has this weirdness that indentation must be respected. And it can be annoying, as differences between spaces and tabs will cause errors. I find it best to work with an programmer's editor that has a "Show Invisibles" option. Under this option, the invisible characters, including space, tab and return, show as a special glyph, so you can see exactly the nature of your white space.


In [5]:
# python --version
# Python 3.7.3


# the most typical def statement:
def hello(x):
    print("hello",end=" ")  # the way to not have a newline
    print(x)

# there is a limited support for a single line

def hello_singleline(x):print("hello",x)

# also lambda's construct the function, without nameing it, but the constructed
# function can be 

hello_lambda = lambda x: print("hello",x)

# now try them (invoking the function)
hello("world")
hello_singleline("world")
hello_lambda("world")

hello world
hello world
hello world


#### If-Else

Here is an example of the if-else written out with full indentation (there are also one-line forms).

In [6]:
# if else a great thing to know about
# there is also one line versions without indentation, we skip this

def sometimes(x):
    if x>0: 
        return x-1
    else: 
        return 0
    
print(sometimes(3))
print(sometimes(sometimes(3)))
print(sometimes(sometimes(sometimes(3))))
print(sometimes(sometimes(sometimes(sometimes(3)))))

# a short form that's useful
def sometimes(x):
    return x-1 if x>0 else 0

print(sometimes(3))
print(sometimes(sometimes(3)))
print(sometimes(sometimes(sometimes(3))))
print(sometimes(sometimes(sometimes(sometimes(3)))))


2
1
0
0
2
1
0
0


#### Combing if-else and defs: recursion

The variety of computer languages hides the fact that in essence all programming languages are equally powerful. What can be done in one, can be done in another. As an example, while a looping construct is present in all computing languages, and seemingly essential, looping can be created from if-else and function calls. Rather than returning up on the code flow, the function calls another invocation of itself.

This is probably the first non-trivial program. It counts down from a given number, until the count is zero. Then it prints BOOM! and halts. The recursive structure is: 

* if the given number x is greater than zero, print it and then consider the problem on x-1
* else print BOOM! and that's it


In [13]:
# repeating thing over by recursion
# this is an interesting program for learning
# about value binding, and nesting of dictionaries

def boom(x):
    if x>0:
        print(x)
        boom(x-1)
    else:
        print("BOOM!")
        
print("There's no place like BOOM!\n")
boom(10)


def boom_stacking(x,level):
    
    def indent(i):
        print(" "*level*2,end=" ")
    
    if x>0:
        indent(level)
        print(x)
        boom_stacking(x-1,level+1)
        indent(level)
        print(x)
    else:
        indent(level)
        print(0)

print("\n\nBOOM showing the recovery of stacked values.\n")
boom_stacking(10,0)


There's no place like BOOM!

10
9
8
7
6
5
4
3
2
1
BOOM!


BOOM showing the recovery of stacked values.

 10
   9
     8
       7
         6
           5
             4
               3
                 2
                   1
                     0
                   1
                 2
               3
             4
           5
         6
       7
     8
   9
 10


#### Namespaces, scoping, and the Excution Model

Recursion and recursive solutions lead to a discussion of how names are associated with values. In recursion, a mystery arises: how can the same name, in this case x, appear in the text once yet seem to contain multiple values during the recursion. The mystery is explained by the excution model that creates a _namespace_ with each entry to the function, and another when the function is entered again, and another, as often as required for the recursion. A namespace is a collection of _bindings_, and a binding is a name-value pair. 

When a name is encountered, a search is made for a binding of the name in one or more namespaces. The namespace searched depends both on runtime events, according to the _execution model_, and static text of the program, according to the _scope_. The static text analysis is called _lexical scoping_.

When the define is evaluted, a function object is created and bound the given name. And the lexical scope of the names are determined. The names inside the function, including the names of the arguments to the function, belong to the _local scope_ of that function. (N.B. Unless it is determined that it belongs to an enclosing scope, or an explicitly stated scope.) Names that occur outside of any define (or class definition, c.f.) are in the _global scope_.

Example: In boom_stacking, boom_stacking is in the _global scope_ that contains all top-level defines. Names x, level and indent are in the _local scope_ of the function boom_stacking, and the name i is in the local scope of the function indent. 

When a function is entered, a namespace is created for the bindings of the names in the local scope. The names of the arguments are entered into then namespace, bound to the values they receive from the call. As names are encountered in the scope, they are either bound with initial values, rebound with new values, or the value bound to the name is used. If a name is used before bound, an UnboundLocalError is raised.

When the function exits, its namespace is forgotten. 

A binding cannot occur before a namespace exists to contain the binding. The global namespace is created at the start of the Python session, so as global names are encountered, bindings are entered. The _lifetime_ of the global namespace is co-extensive with the session run. However, a local namespace only exists while the function is executing. Its lifetime is co-extensive with the run of the function. While a name's local scope is determined when the define is encountered, a binding for the name must wait until the function is run and a namespace created.

The mystery of recursion is then solved as follows. With each recursive call the machine model creates a fresh namespace. The names in local scope are entered in to this namespace as they encountered (including the formals which appear to be encountered immediately). The previous namespace continues to exist, but its name bindings are not used. When the function returns the machine model dissolves the recursive namespace and the previous name bindings are recovered.

<pre>
   boom(10) -- | n : 10 |
     |
     +-- boom(9) -- | n : 9 |
          |
          +-- boom(8) -- | n : 8 |
               |
               etc ...
</pre>




In [18]:

def boom(n): # in the global namespace
    
    def boom(n): # in the local namespace
        if n>0:
            print(n)
            boom(n-1)  # uses the local
        else:
            print("BOOM!")
            
    boom(n)
    print("Done!")
            
boom(10)


def boom1(n):   # boom1 is in the global namesapce 
    if n>0:
        print(n)
        boom1(n-1)  # this name bounds to the global namespace
    else:
        print("BOOM!")

def wrapper(f,n):
    
    def boom1(n):   # this is in wrapper local namespace
        print("DUD!")
        
    f(n)    # when f, which is boom1, uses name boom1, it will refer to the boom1 in the 
            # global directory, no the boom1 in the local namespace, 
            # because the scope is lexical, determined when boom1 was defined

wrapper(boom1,10)

10
9
8
7
6
5
4
3
2
1
BOOM!
Done!
10
9
8
7
6
5
4
3
2
1
BOOM!


#### Choosing the global namespace

There are cases where a name's scope is ambiguous. A name inside a define might wish to be in global scope. Also, defines can nest, and an inner define might use a name in the scope of an outer define. The name might be in the scope of the inner define, the outer define, or in global scope.

Example. This occurs in our boom program, as the name boom is in global scope, but the recursive call uses the name boom possibly in local scope. However, it can't be the case that the boom name inside the boom function is in local scope, as boom is not a bound name in the local namespace. So that occurnace of the name should refer to the global scope, and meaning is that the boom function shoule be re-entered.

We consider the simple case of a name in both local and global scope. Here is the rule by which this is decided:

> If a variable is ambiguously in both a global and local scope, if the variable is ever written into, the scope shall be local and the binding in the local namespace is used; if the variable is always and only read, the scope shall be global and the global namespace is used. The formal parameters if a function are in local scope.

It is to be emphasized that the scoping is determined when the define is encountered. Python uses _lexical scoping_. That means that the scope of a name is determined when the define is evaluated. The alternative, _dynamic scoping_ is a bit obscure, but not unused &mdash; it is the method used by macro packages, and distinguishes macro languages from programming languages.

In [29]:
# lexical binding

i = 4  # entered into the global namespace

def a():
    i = 6   # entered into the local namespace
    print(i)  # uses the i in the local namespace
    return b()  # even though there is an i in the local namespace, the scope of i in b remains global

def b():
    return i # scopes to the global i
    
print("calling b:",b())
print("calling a:",a())


calling b: 4
6
calling a: 4


#### Implicit choice of scope

Languages such as C require the declaration of names, and that guides scoping. Python refuses to require this, and so scoping is implicit. The rule is uses is as what we said above. The global namespace, in effect, is treated as immutable. Therefore attempts to write a global are avoided by making names that attempt such to be local. The scope of the name is constant throughout the def. It does not start global until such a point when it becomes local. Here are examples.

_N.B. The error is called UnboundLocalError. The entering of a name into a namespace is called the binding of the name. So an "unset" variable is because it is unbound._

In [36]:
i = 6  # i is entered into the global namespace

def h1(x): 
    i = i + 1  # because i is set, it is local; 
               # but then it is being used before it is set, an error

def h2(x): # this demonstrates when javascript calls "variable hoisting" 
    j = i  # while i was not yet set ...
    i = j  # it is here, so i has local scope
           # so in the line above, i is used before being set, an error

try:
    h1(1)
except UnboundLocalError:
    print("h1: I told you it wouldn't work!")
    
try:
    h2(1)
except UnboundLocalError:
    print("h2: I told you it wouldn't work!")
   
# Note that python evaluates the def, and places an entry or it in the global namespace, so we
# can call the function. the error is at runtime, when the scope has to actually find a namespace

h1: I told you it wouldn't work!
h2: I told you it wouldn't work!


#### Advanced scoping discussion

Python has keywords <code>global</code> and <code>nonlocal</code> to determine the _scoping_ of a variable, and to permit rebinding of names (value updates) when the scope is not strictly local. 

The global declaration for a name forces global scope for the name, even if the name will recieve a value. This allows for the rebinding of a global name when it appears inside a function.

The nonlocal declaration of a name forces local scope, but not in the innermost scope when defs nest. The binding will be determined lexically, that is, by the text of the program, to some containing namespace. If there is no such, the define will fail as a syntax error.


_N.B: The heavy reading here is at Python.org the <a href="https://docs.python.org/3/reference/executionmodel.html">Execution Model</a>._


_N.B: The heavy reading here is <a href="https://www.python.org/dev/peps/pep-3104/">PEP 3104</a>_


In [39]:

# which values change ?

i_in_global = 101
j_in_global = 102
k_in_global = 103

i_in_nonlocal = 104
j_in_nonlocal = 105
k_in_nonlocal = 105


def f():
    i_in_nonlocal = 201
    j_in_nonlocal = 202
    k_in_nonlocal = 203

    
    def g():
        global i_in_global
        nonlocal j_in_nonlocal
        i_in_global = 111
        j_in_global = 112
        k_in_global = 113
        i_in_nonlocal = 211
        j_in_nonlocal = 212
        k_in_nonlocal = 213
        
        print("\n\t\tin g:")
        print("\t\tsaid to be global:", i_in_global, j_in_global, k_in_global)
        print("\t\tsaid to be nonlocal:", i_in_nonlocal, j_in_nonlocal, k_in_nonlocal)

    print("\n\tin f before call to g:")
    print("\tsaid to be global:", i_in_global, j_in_global, k_in_global)
    print("\tsaid to be nonlocal:", i_in_nonlocal, j_in_nonlocal, k_in_nonlocal)

    g()
    print("\n\tin f after call to g:")
    print("\tsaid to be global:", i_in_global, j_in_global, k_in_global)
    print("\tsaid to be nonlocal:", i_in_nonlocal, j_in_nonlocal, k_in_nonlocal)


print("\nin global before call to f:")
print("said to be global:", i_in_global, j_in_global, k_in_global)
print("said to be nonlocal:", i_in_nonlocal, j_in_nonlocal, k_in_nonlocal)

f()

print("\nin global after call to g:")
print("said to be global:", i_in_global, j_in_global, k_in_global)
print("said to be nonlocal:", i_in_nonlocal, j_in_nonlocal, k_in_nonlocal)
         
def h():
    nonlocal i_in_global
    return i_in_global



in global before call to f:
said to be global: 101 102 103
said to be nonlocal: 104 105 105

	in f before call to g:
	said to be global: 101 102 103
	said to be nonlocal: 201 202 203

		in g:
		said to be global: 111 112 113
		said to be nonlocal: 211 212 213

	in f after call to g:
	said to be global: 111 102 103
	said to be nonlocal: 201 212 203

in global after call to g:
said to be global: 111 102 103
said to be nonlocal: 104 105 105


SyntaxError: no binding for nonlocal 'i_in_global' found (<ipython-input-39-e42ce35dd383>, line 55)

### Exercises:

Create recursive functions for the fibbonacci series, 

 f_i = f_{i-1} + f_{i-2}, i>=2
 f_0 = 0
 f_1 = 1
 
the binomial series,

  (n choose k) = (n-1 choose k) + (n-1 choose k-1), when 1=<k<=n-1
  (n choose 0) = (n choose n) = 1 all n>=0

In [10]:

# fix my code!! 

def fib(x):
    if x==0:
        return 0
    if x==1:
        return 1
    return -1

# fix my code!! 

def choose(n,k):
    if k>=1 and k<=n-1:
        return 0
    return -1


#### test programs ....

def test_fib():
    answers = [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
    i = 1
    for ans in answers:
        if fib(i)!=ans:
            print("broken!")
            return
        i = i + 1
    print("correct!")


def test_choose():
    answers = [[1,1],[1,2,1],[1,3,3,1],[1,4,6,4,1],[1,5,10,10,5,1]]
    n = 1
    k = 0
    for ans in answers:
        for v in ans:
            if choose(n,k)!=v:
                print("broken!")
                return
            k = k + 1
        k = 0
        n = n + 1
    print("correct!")

#### run the tests
        
test_fib()
test_choose()

broken!
broken!
